In [ ]:
import os.path
import shutil
from urllib.request import urlretrieve
import gc

import numpy as np
import numpy.ma as ma

from scipy import ndimage
from scipy import fftpack
from scipy.stats import norm
from scipy.interpolate import interp1d, PchipInterpolator
from scipy.io import wavfile
from scipy.signal import savgol_filter
from scipy.signal import ricker

import warnings
warnings.filterwarnings('ignore', '.*output shape of zoom.*')
warnings.filterwarnings('ignore', '.*non-tuple sequence for multidimensional indexing is deprecated.*')

import matplotlib
try:
    if test_run:
        %matplotlib inline
except:
    matplotlib.use('agg')

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.animation as animation

from IPython.display import clear_output

from astropy.table import Table
from astropy.io import fits
from astropy.cosmology import Planck13
from astropy.convolution import convolve, convolve_fft, Gaussian2DKernel, Tophat2DKernel
from astropy.visualization import AsymmetricPercentileInterval

#cat_dir = '%s/cluster_catalogue' % project_dir

tmp_dir = '%s/temp_files' % project_dir
img_dir = '%s/image_files' % tmp_dir
aud_dir = '%s/audio_files' % tmp_dir
vid_dir = '%s/video_files' % tmp_dir

%run './python_libs/utils_catalogues.ipynb'
%run './python_libs/utils_wise.ipynb'
%run './python_libs/utils_images.ipynb'
%run './python_libs/utils_frames.ipynb'
%run './python_libs/utils_effects.ipynb'
%run './python_libs/utils_sound.ipynb'
%run './python_libs/utils_video.ipynb'


#=================================================================================================================
# Directories utilities:
#=================================================================================================================

#-----------------------------------------------------------------------------------------------------------------
def create_folders(new_project=False):
    """
    Creates the folders needed to run the code.
    """
    
    if new_project and os.path.isdir(tmp_dir):
        shutil.rmtree(tmp_dir)
        os.makedirs(tmp_dir)
    elif not os.path.isdir(tmp_dir):
        os.makedirs(tmp_dir)
        
    for x_dir in [img_dir, aud_dir, vid_dir, vid_dir+'/AUD', vid_dir+'/VID']:
        if not os.path.isdir(x_dir):
            os.makedirs(x_dir)
            
    
#=================================================================================================================
# Configuration utils
#=================================================================================================================

#-----------------------------------------------------------------------------------------------------------------
class Configuration(object):
    """
    Defines configuration.
    """
    
    def __init__(self):
        
        #--------------------------------------------------------------------------
        ## Catalogue Selection:

        # Richness minimum cutoff: [rich_cutoff = 50.]
        self.rich_cutoff = 40.
        # Minimum redshift cutoff: [z_min = 0.20]
        self.z_min = 0.20
        # Maximum redshift cutoff: [z_max = 0.35]
        self.z_max = 0.35

        #--------------------------------------------------------------------------
        ## Images config:

        # Size of the images downloaded from WISE (not final video size): [img_pix = 800]
        self.img_pix = 1000
        # Size in Mpc of the images shown in the video: [img_mpc = 1.5]
        self.img_mpc = 2.5

        #--------------------------------------------------------------------------
        ## Interpolation config:

        # Use logarithmic interpolation? (True recommended): [log_interp = True]
        self.log_interp = True

        #--------------------------------------------------------------------------
        ## Video config:

        #self.video_format = '4:3'
        self.video_format = '16:9'
        
        # Final video size in pixels:
        #self.video_pixx = 1920 # In 16:9 format, 1920 implies frames of 1920x1080
        self.video_pixx = 1280 # In 16:9 format, 1280 implies frames of 1280x720
        self.video_pixblackstripes = 60
        
        # Specify screen dpi:
        self.my_dpi = 154.2*(700./1080.)
        
        # ffmpeg crf compression quality:
        self.crf_quality = 18

    #--------------------------------------------------------------------------
    def set_video_info(self, fps, fpi, video_duration):
        
        if self.video_format == '4:3':
            self.video_ratio = 4./3.
        if self.video_format == '16:9':
            self.video_ratio = 16./9.
        if self.video_format == '1:1':
            self.video_ratio = 1.
                    
        self.video_pixy = np.int(1.*self.video_pixx/self.video_ratio) - 2*self.video_pixblackstripes
        
        self.fps = fps # Frames per second
        self.fpi = fpi # Frames per image
        self.video_duration = video_duration # Duration of video
        
        self.n_images = np.int(np.ceil(self.video_duration*self.fps/fpi)) + 1
        self.t_per_image = (1.*self.fpi)/self.fps
        
        self.n_frames = np.int(np.ceil(self.video_duration*self.fps))
        self.t_per_frame = 1./self.fps
                
    #--------------------------------------------------------------------------
    def display_video_info(self):
        
        print('\nVideo information:')
        print()
        print('fpi: %i' % self.fpi)
        print('Duration: %.03fs' % self.video_duration)
        print()
        print('nº of images:   %i' % self.n_images)
        print('time per image: %.03fs' % self.t_per_image)
        print('nº of frames:   %i' % self.n_frames)
        print('time per frame: %.03fs' % self.t_per_frame)
        print()
        print('time offset start:   %.03fs' % self.t_offset_start)
        print('frames offset start: %i' % self.frames_offset_start)
        print()
        print('time black start:   %.03fs' % self.t_black_start)
        print('frames black start: %i' % self.frames_black_start)
        print()
        print('Video resolution:    %ix%i' % (self.video_pixx, np.int(1.*config.video_pixx/config.video_ratio)))
        print('Display resolution:  %ix%i' % (self.video_pixx, self.video_pixy))
        print('Video black stripes: %i pix - %i pix' % (self.video_pixblackstripes, self.video_pixblackstripes))
        print()
        print('*'*20)

    #--------------------------------------------------------------------------
    def set_audio_info(self, audio_rate, n_audio_samples):
        
        self.audio_rate = audio_rate
        self.n_audio_samples = n_audio_samples
        self.audio_duration = 1.*n_audio_samples/audio_rate
        
    #--------------------------------------------------------------------------
    def set_t_offset_start(self, t_offset_start):
        self.t_offset_start = t_offset_start
        self.frames_offset_start = np.int(np.ceil(t_offset_start/self.t_per_frame))
        
    #--------------------------------------------------------------------------
    def set_t_black_start(self, t_black_start):
        self.t_black_start = t_black_start
        self.frames_black_start = np.int(np.ceil(t_black_start/self.t_per_frame))
            
    #--------------------------------------------------------------------------
    def set_intro(self, t_intro_off1, t_intro_on, t_intro_off2):
        self.t_intro_off1 = t_intro_off1
        self.t_intro_on = t_intro_on
        self.t_intro_off2 = t_intro_off2
        self.t_intro = t_intro_off1 + t_intro_on + t_intro_off2
        
    #--------------------------------------------------------------------------        
    def set_credits(self, t_credits_off1, t_credits_on, t_credits_off2):
        self.t_credits_off1 = t_credits_off1
        self.t_credits_on = t_credits_on
        self.t_credits_off2 = t_credits_off2
        self.t_credits = t_credits_off1 + t_credits_on + t_credits_off2
        
    #--------------------------------------------------------------------------
    def set_rich_cutoff(self, rich_cutoff):
        self.rich_cutoff = rich_cutoff

    def set_z_min(self, z_min):
        self.z_min = z_min

    def set_z_max(self, z_max):
        self.z_max = z_max

    def set_img_pix(self, img_pix):
        self.img_pix = img_pix

    def set_img_mpc(self, img_mpc):
        self.img_mpc = img_mpc

    def set_wise_band(self, wise_band):
        self.wise_band = wise_band

    def set_fpc(self, fpc):
        self.fpc = fpc

    def set_log_interp(self, log_interp):
        self.log_interp = log_interp

    def set_fsigma_base(self, fsigma_base):
        self.fsigma_base = fsigma_base

    def set_fsigma_eff(self, fsigma_eff):
        self.fsigma_eff  = fsigma_eff

    def set_fps(self, fps):
        self.fps = fps

    def set_video_pix(self, video_pix):
        self.video_pixx = video_pix
        
    def set_video_format(self, video_format):
        self.video_format = video_format
          
    def set_video_pixblackstripes(self, video_pixblackstripes):
        self.video_pixblackstripes = video_pixblackstripes

    def set_my_dpi(self, my_dpi):
        self.my_dpi = my_dpi
        
    def set_crf_quality(self, crf_quality):
        self.crf_quality = crf_quality

    def set_video_version(self, video_version):
        self.video_version = video_version

    def set_color_map(self, color_map):
        self.color_map = color_map

    def set_log_scale(self, log_scale):
        self.log_scale = log_scale

    def set_v_max(self, v_max):
        self.v_max = v_max
    